# 如何使用向量存储作为检索器

向量存储检索器是一种使用[向量存储](/docs/concepts/vectorstores/)来检索文档的[检索器](/docs/concepts/retrievers/)。它是一个围绕向量存储类的轻量级包装器，使其符合检索器[接口](/docs/concepts/runnables/)。
它利用向量存储实现的搜索方法，如相似性搜索和MMR，来查询向量存储中的文本。

本指南将涵盖：

1. 如何从向量存储实例化检索器；
2. 如何为检索器指定搜索类型；
3. 如何指定额外的搜索参数，如阈值分数和 top-k。

## 从向量存储创建检索器

您可以使用向量存储的 [.as_retriever](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html#langchain_core.vectorstores.base.VectorStore.as_retriever) 方法从向量存储构建检索器。让我们通过一个例子来介绍。

首先，我们实例化一个向量存储。我们将使用一个内存中的 [FAISS](https://python.langchain.com/api_reference/community/vectorstores/langchain_community.vectorstores.faiss.FAISS.html) 向量存储：

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("state_of_the_union.txt")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(texts, embeddings)

然后我们可以实例化一个检索器：

In [2]:
retriever = vectorstore.as_retriever()

这将创建一个检索器（具体来说是 [VectorStoreRetriever](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStoreRetriever.html)），我们可以像平常一样使用它：

In [3]:
docs = retriever.invoke("what did the president say about ketanji brown jackson?")

## 最大边际相关性检索
默认情况下，向量存储检索器使用相似性搜索。如果底层向量存储支持最大边际相关性搜索，您可以将其指定为搜索类型。

这实际上指定了底层向量存储使用的什么方法（例如，`similarity_search`、`max_marginal_relevance_search` 等）。

In [4]:
retriever = vectorstore.as_retriever(search_type="mmr")

In [5]:
docs = retriever.invoke("what did the president say about ketanji brown jackson?")

## 传递搜索参数

我们可以使用 `search_kwargs` 将参数传递给底层的向量存储的搜索方法。

### 相似度分数阈值检索

例如，我们可以设置一个相似度分数阈值，只返回分数高于该阈值的文档。

In [6]:
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5}
)

In [7]:
docs = retriever.invoke("what did the president say about ketanji brown jackson?")

### 指定 top k

我们也可以限制检索器返回的文档数量 `k`。

In [8]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [9]:
docs = retriever.invoke("what did the president say about ketanji brown jackson?")
len(docs)

1